# To test virtual scan module

In [1]:
from VirtualScanTools import take_scan_virtual
import matplotlib.pyplot as plt
import numpy as np
import datasets as ds

path = "C:/Users/Maxwell/Imperial College London/complex nanophotonics - PH - 20241101_sanity checks"

In [2]:
# check if code running on GPU
import tensorflow as tf
from tensorflow.python.client import device_lib
print('-'*20)
print('List local devices:')
print(device_lib.list_local_devices())
print('\nList GPU devices:')
print(tf.config.list_physical_devices('GPU'))
print('\nRunning on cuda:', tf.test.is_built_with_cuda())

--------------------
List local devices:
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17888281333155701841
xla_global_id: -1
]

List GPU devices:
[]

Running on cuda: False


## Testing model before mnist (overfitted)

In [2]:
from tensorflow.keras.models import load_model
model = load_model("M:/MSci_neuromorphic-dt-main/history/model_updated_size3.keras")

#### Ham95 data
formated in the same was as the experiment

In [6]:
ham95_images = ds.load(path + "/source_images/ham95_shuffled.ds").raw
ham95_images = np.rot90(ham95_images, -1, axes=(1, 2))

rot = 0
ham95_images_rot0 = np.rot90(ham95_images, -1+rot, axes=(1, 2))

rot = 1
ham95_images_rot1 = np.rot90(ham95_images, -1+rot, axes=(1, 2))

experimental spectrums

In [7]:
import glob
expr_spectrum_paths_rot0 = sorted(glob.glob(path + "/data/ham95_shuffled_gTrue_rot0_/*[0-9].ds"))
expr_spectrum_paths_rot1 = sorted(glob.glob(path + "/data/ham95_shuffled_gTrue_rot1_/*[0-9].ds"))

In [8]:
from tools import spectrum_retrival
expr_spectrum_rot0 = spectrum_retrival(expr_spectrum_paths_rot0)
expr_spectrum_rot1 = spectrum_retrival(expr_spectrum_paths_rot1)

predicted spectrums

In [11]:
power = 310
mult = 4
pad_level = 166
model_path = "M:/MSci_neuromorphic-dt-main/history/model_updated_size3.keras"

pred_spectrum_rot0 = take_scan_virtual(
    ham95_images_rot0, 
    model_path,
    4,
    power,
    pad_level
)

pred_spectrum_rot1 = take_scan_virtual(
    ham95_images_rot1, 
    model_path,
    4,
    power,
    pad_level
)

 99/367 [=======>......................] - ETA: 3:10

KeyboardInterrupt: 

In [10]:
help(take_scan_virtual)

Help on function take_scan_virtual in module VirtualScanTools:

take_scan_virtual(data, model_path, mult, power, padding_value=0, invert=False)
    This script takes in data like in the physical system, but loads a keras model and does the prediction    
    ----------
    Parameters
    ----------
    data: ndarray
        This is the raw image data that is to be fed into the system images stacked on axis=0 and must be square.
        
    model_path: str
        Path directory to where the model is stored.
        
    mult: int 
        Same as in take_scan() from experiment
            Images have a multiplication factor that scales up the images to fit within a total boarder of 532
            This code finds the length for experiment then scales it for the model
            
    power: int
        power of the laser in the experiment.
        
    padding_value : int, optional
        If images do not fill the full space, padding is added usualy the modal pixel strength in the da